In [1]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
# Create a file directory
file_dir = '/Users/rfnichol/OneDrive - COOPER TIRE & RUBBER COMPANY/Personal/Data Analytics Boot Camp/Module_8_ETL/Module_8_files/Movies_ETL'
# file_dir = '/Users/rfnichol/Movies_ETL'

In [3]:
# Open the json file
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [4]:
# How many records were pulled in? Use the len function
len(wiki_movies_raw)

7311

In [5]:
# First 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [6]:
# Last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [7]:
# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [8]:
# Since the Kaggle data is already in flat-file formats, we'll just pull them into Pandas DataFrames directly 
# with the following code
kaggle_metadata_df = pd.read_csv(f'{file_dir}/archive/movies_metadata.csv', low_memory=False)
ratings_df = pd.read_csv(f'{file_dir}/archive/ratings.csv')

In [9]:
# View the dataframe using the head method for any correctins or issues
kaggle_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
# View the dataframe using the head method for any correctins or issues
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [11]:
# errors can still occur in the middle of the file, so the best practice is to sample a handful of rows randomly using the 
# sample() method. For a DataFrame called df, df.sample(n=5) will show five random rows from the dataset. 
kaggle_metadata_df.sample(n=5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
44333,False,NaN,0,[],NaN,424971,tt4072742,en,In View,Tormented by guilt over the death of her famil...,...,2016-11-20,0.0,93.0,[],Released,NaN,In View,False,0.0,0.0
31644,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,326139,tt2782868,en,Eadweard,"In the second half of the 19th century, Eadwea...",...,2015-02-28,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Eadweard,False,7.8,4.0
18434,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,97394,tt1263704,en,Stanley Kubrick's Boxes,"A few years after his death, the widow of Stan...",...,2008-07-15,0.0,48.0,[],Released,NaN,Stanley Kubrick's Boxes,False,6.8,15.0
44200,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,339445,tt0219765,pt,Henry Miller Asleep & Awake,"Filmed when the Henry Miller was 81, is a voya...",...,1975-02-14,0.0,35.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Henry Miller Asleep & Awake,False,0.0,0.0
31316,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,85110,tt0058025,en,Dog Star Man: Part I,"From a murky landscape, a wooded mountain emer...",...,1962-01-01,0.0,30.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,Dog Star Man: Part I,False,6.5,18.0


In [12]:
# errors can still occur in the middle of the file, so the best practice is to sample a handful of rows randomly using the 
# sample() method. For a DataFrame called df, df.sample(n=5) will show five random rows from the dataset. 
ratings_df.sample(n=5)

,userId,movieId,rating,timestamp
8878634,91531,2427,4.5,1279890591
22360027,232042,111113,2.5,1464197875
997619,10162,552,4.0,901191357
21988840,228408,2315,5.0,913325700
15951967,165926,108932,3.0,1455409345


In [13]:
# see all of the column names that were imported.
kaggle_metadata_df.columns.tolist() 

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [14]:
# codify our JSON data by restricting it to only those entries that have a director and an IMDb link. We can do this 
# with a list comprehension.
kaggle_movies = [movie for movie in kaggle_metadata_df
                   if 'original_title' in movie]
 #                  and 'imdb_id' in movie]
len(kaggle_movies)

1

In [15]:
# Create a dataframe for the wiki_movies_raw file
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [16]:
# view the wifi_movies_df
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# view the wifi_movies_df using the sample function
wiki_movies_df.sample(n=10)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
1562,https://en.wikipedia.org/wiki/Citizen_Ruth,1996.0,https://www.imdb.com/title/tt0115906/,Citizen Ruth,Alexander Payne,"[Cary Woods, Cathy Konrad, Andrew Stone, Micha...",NaN,NaN,NaN,Laura Dern,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4815,https://en.wikipedia.org/wiki/College_Road_Trip,2008.0,https://www.imdb.com/title/tt0997047/,College Road Trip,Roger Kumble,"[Andrew Gunn, Raven-Symoné, (executive produce...",NaN,NaN,NaN,"[Raven-Symoné, Martin Lawrence, Donny Osmond]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6068,"https://en.wikipedia.org/wiki/Grow_Up,_Tony_Ph...",2013.0,https://www.imdb.com/title/tt2630096/,"Grow Up, Tony Phillips",Emily Hagins,NaN,NaN,NaN,NaN,"Trevon D. Anderson, Caleb Barwick, Emmet Barwick",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1735,https://en.wikipedia.org/wiki/Mojave_Moon,1996.0,https://www.imdb.com/title/tt0117070/,Mojave Moon,Kevin Dowling,Matt Salinger,NaN,NaN,NaN,"[Danny Aiello, Anne Archer, Angelina Jolie, Mi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,https://en.wikipedia.org/wiki/Home_Alone_2:_Lo...,1992.0,https://www.imdb.com/title/tt0104431/,Home Alone 2: Lost in New York,Chris Columbus,John Hughes,NaN,NaN,"[Characters, by John Hughes]","[Macaulay Culkin, Joe Pesci, Daniel Stern, Joh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,https://en.wikipedia.org/wiki/Green_Card_(film),1990.0,https://www.imdb.com/title/tt0099699/,Green Card,Peter Weir,Peter Weir,NaN,NaN,NaN,"[Gérard Depardieu, Andie MacDowell]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2232,https://en.wikipedia.org/wiki/An_Alan_Smithee_...,1998.0,https://www.imdb.com/title/tt0118577/,An Alan Smithee Film:Burn Hollywood Burn,"[Arthur Hiller, (as, Alan Smithee, )]",Ben Myron,NaN,NaN,NaN,"[Ryan O'Neal, Coolio, Chuck D, Eric Idle, Sylv...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3875,https://en.wikipedia.org/wiki/Doom_(film),2005.0,https://www.imdb.com/title/tt0419706/,Doom,Andrzej Bartkowiak,Lorenzo di Bonaventura,"[David Callaham, Wesley Strick]",NaN,"[Doom, by, id Software]","[Karl Urban, Rosamund Pike, Razaaq Adoti, Dway...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3529,https://en.wikipedia.org/wiki/Puppet_Master:_T...,2003.0,https://www.imdb.com/title/tt0390393/,Puppet Master: The Legacy,Charles Band,"[Charles Band, Kurt Iswarienko, Matt Wolpert]",NaN,NaN,NaN,"[Jacob Witkin, Kate Orsini]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5991,https://en.wikipedia.org/wiki/Can_a_Song_Save_...,2013.0,https://www.imdb.com/title/tt1980929/,Begin Again,John Carney,"[Anthony Bregman, Tobin Armbrust, Judd Apatow]",NaN,NaN,NaN,"[Keira Knightley, Mark Ruffalo, Hailee Steinfe...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# see all of the column names that were imported from wiki_movies.
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [19]:
# Create a list comprehension with the filter expression we created and save that to an intermediate variable wiki_movies.
# See how many movies are in wiki_movies with the len() function.
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [20]:
# Create a function to clean up the data
# 1 Name
# 2 Parameters
# 3 Code block
# 4 Return value

# Scope 
# Inside the code block of a function, we can use variables that were created outside the function and 
# initialize new variables inside the function.
# This is called the "scope" of the variables:

#    Variables created outside the function are called global variables.
#    New variables created inside the function are local variables.
#    The hierarchy of variables is called the scope.

# For example consider the following code
x = 'global value'

def foo():
    x = 'local value'
    print(x)

foo()
print(x)


local value
global value


In [21]:
# we have to be careful with variables we send to a function. Any data structure can be sent as a parameter to 
# a function, including dicts, lists, and tuples, but we must be careful with mutable objects.
my_list = [1,2,3]
def append_four(x):
    x.append(4)
append_four(my_list)
print(my_list)

[1, 2, 3, 4]


In [22]:
# Lambda functions which is the most stripped down we can make. Lambda functions are written in one line and automatically 
# return a value. without using the return keyword. Lambda functions have no name and are also known as 
# "anonymous functions."
# Bsic syntax: lambda arguments: expression
# lambda x: x * x
square = lambda x: x * x
square(5)

25

In [23]:
# Create a function to clean up the data
# 1 Name
# 2 Parameters
# 3 Code block
# 4 Return value

# Iteration 1
# def clean_movie(movie):
#    movie_copy = dict(movie)

# Iteration 2 
# def clean_movie(movie):
#    movie = dict(movie) #create a non-destructive copy

# Iteration 3
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

In [24]:
# Inspection
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7060,https://en.wikipedia.org/wiki/The_Insult_(film),2018.0,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7293,https://en.wikipedia.org/wiki/Capernaum_(film),2018.0,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [26]:
# Go through each of the columns, one by one, and determine which columns hold alternate titles.
# Module Skill Drill wanted you to look at each column and corresponding date to find column that contains alternate title
sorted(wiki_movies_df.columns.tolist())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [27]:
# Alternate way of finding which column contains Alternate Title
wiki_movies_df.sample(n=20)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7179,https://en.wikipedia.org/wiki/Uncle_Drew,2018.0,https://www.imdb.com/title/tt7334528/,Uncle Drew,Charles Stone III,"[Marty Bowen, Wyck Godfrey]",NaN,NaN,NaN,"[Kyrie Irving, Lil Rel Howery, Shaquille O'Nea...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,https://en.wikipedia.org/wiki/Totally_F***ed_Up,1993.0,https://www.imdb.com/title/tt0108366/,Totally Fucked Up,Gregg Araki,"[Gregg Araki, Andrea Sperling]",NaN,NaN,NaN,"[James Duval, Roko Belic, Susan Behshid, Jenee...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,https://en.wikipedia.org/wiki/Swimming_with_Sh...,1994.0,https://www.imdb.com/title/tt0114594/,Swimming With Sharks,George Huang,NaN,NaN,NaN,NaN,"[Kevin Spacey, Frank Whaley, Michelle Forbes, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3696,https://en.wikipedia.org/wiki/Kinsey_(film),2004.0,https://www.imdb.com/title/tt0362269/,Kinsey,Bill Condon,Gail Mutrux,NaN,NaN,NaN,"[Liam Neeson, Laura Linney, Chris O'Donnell, P...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,https://en.wikipedia.org/wiki/Boxing_Helena,1993.0,https://www.imdb.com/title/tt0106471/,Boxing Helena,Jennifer Chambers Lynch,"[Philippe Caland, Carl Mazzocone]",Jennifer Chambers Lynch,Philippe Caland,NaN,"[Julian Sands, Sherilyn Fenn, Bill Paxton, Art...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,https://en.wikipedia.org/wiki/Jurassic_Park_III,2001.0,NaN,Jurassic Park III,Joe Johnston,"[Kathleen Kennedy, Larry Franco]",NaN,NaN,"[Characters, by, Michael Crichton]","[Sam Neill, William H. Macy, Téa Leoni, Alessa...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3521,https://en.wikipedia.org/wiki/Paycheck_(film),2003.0,https://www.imdb.com/title/tt0338337/,Paycheck,John Woo,"[John Woo, John Davis, Terence Chang, Michael ...",NaN,NaN,"["", Paycheck, "", by, Philip K. Dick]","[Ben Affleck, Aaron Eckhart, Uma Thurman, Paul...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7144,https://en.wikipedia.org/wiki/Life_of_the_Part...,2018.0,https://www.imdb.com/title/tt5619332/,Life of the Party,Ben Falcone,"[Ben Falcone, Melissa McCarthy, Chris Henchy]",NaN,NaN,NaN,"[Melissa McCarthy, Gillian Jacobs, Maya Rudolp...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4028,https://en.wikipedia.org/wiki/Traci_Townsend,2005.0,https://www.imdb.com/title/tt0457505/,Traci Townsend,Craig Ross Jr.,"[Bobby Thompson, Cheryl Bedford, Fred Lewis, M...",NaN,NaN,NaN,"[Jazsmin Lewis, Mari Morrow, Richard T. Jones,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200,https://en.wikipedia.org/wiki/Ulee%27s_Gold,1997.0,https://www.imdb.com/title/tt0120402/,Ulee's Gold,Victor Nuñez,Jonathan Demme,NaN,NaN,NaN,"[Peter Fonda, Patricia Richardson, Jessica Bie...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Create a function to handle the alternative titles
# 1 Make an empty dict to hold all of the alternative titles. 
# 2 Loop through a list of all alternative title keys:
# 2a   Check if the current key exists in the movie object.
# 2b   If so, remove the key-value pair and add to the alternative titles dict.
# 3 After looping through every key, add the alternative titles dict to the movie object.

# 1 Make an empty dict to hold all of the alternative titles.   
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    return movie












In [29]:
# 2 Loop through a list of all alternative title keys:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        return movie        

In [30]:
# 2a   Check if the current key exists in the movie object.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:

            return movie

In [31]:
# 2b   If so, remove the key-value pair and add to the alternative titles dict.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)

    return movie

In [32]:
# 3 After looping through every key, add the alternative titles dict to the movie object.
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [33]:
# make a list of cleaned movies with a list comprehension:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

# Set wiki_movies_df to be the DataFrame created from clean_movies, and print out a list of the columns.
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'McCune–Reischauer',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [34]:
# Extract the IMDb ID from the IMDb link. Use regular expression (regex).
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

7076
7033


,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Preceded by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


In [35]:
# Skill drill. Howe many null values are in each column?
wiki_movies_df.isnull().sum()

url                          0
year                         0
imdb_link                    0
title                        1
Based on                  4852
Starring                   184
Narrated by               6752
Cinematography             691
Release date                32
Running time               139
Country                    236
Language                   244
Budget                    2295
Box office                1548
Director                     0
Distributor                357
Editor(s)                  548
Composer(s)                518
Producer(s)                202
Production company(s)     1678
Writer(s)                  199
Genre                     6923
Original language(s)      6875
Original network          6908
Executive producer(s)     6936
Production location(s)    6986
Picture format            6969
Audio format              6972
Voices of                 7031
Followed by               7024
Created by                7023
Preceded by               7023
Suggeste

In [36]:
# Skill drill. How many null values are in each column? Alternative method
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4852],
 ['Starring', 184],
 ['Narrated by', 6752],
 ['Cinematography', 691],
 ['Release date', 32],
 ['Running time', 139],
 ['Country', 236],
 ['Language', 244],
 ['Budget', 2295],
 ['Box office', 1548],
 ['Director', 0],
 ['Distributor', 357],
 ['Editor(s)', 548],
 ['Composer(s)', 518],
 ['Producer(s)', 202],
 ['Production company(s)', 1678],
 ['Writer(s)', 199],
 ['Genre', 6923],
 ['Original language(s)', 6875],
 ['Original network', 6908],
 ['Executive producer(s)', 6936],
 ['Production location(s)', 6986],
 ['Picture format', 6969],
 ['Audio format', 6972],
 ['Voices of', 7031],
 ['Followed by', 7024],
 ['Created by', 7023],
 ['Preceded by', 7023],
 ['Suggested by', 7032],
 ['alt_titles', 7012],
 ['Recorded', 7031],
 ['Venue', 7032],
 ['Label', 7031],
 ['Animation by', 7031],
 ['Color process', 7032],
 ['McCune–Reischauer', 7031],
 ['imdb_id', 0]]

In [37]:
# How can you make a list of columns with less than 90% null values. 
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']

In [38]:
# To ensure we hae the correct data types, review the data types in the dataframe
wiki_movies_df.dtypes

url                       object
year                       int64
imdb_link                 object
title                     object
Based on                  object
Starring                  object
Narrated by               object
Cinematography            object
Release date              object
Running time              object
Country                   object
Language                  object
Budget                    object
Box office                object
Director                  object
Distributor               object
Editor(s)                 object
Composer(s)               object
Producer(s)               object
Production company(s)     object
Writer(s)                 object
Genre                     object
Original language(s)      object
Original network          object
Executive producer(s)     object
Production location(s)    object
Picture format            object
Audio format              object
Voices of                 object
Followed by               object
Created by

In [39]:
# start on the box office data, make a data series that drops missing values with the following
box_office = wiki_movies_df['Box office'].dropna()

In [40]:
# Rerun to see impact of dropna(). How many null values are in each column? Alternative method
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4852],
 ['Starring', 184],
 ['Narrated by', 6752],
 ['Cinematography', 691],
 ['Release date', 32],
 ['Running time', 139],
 ['Country', 236],
 ['Language', 244],
 ['Budget', 2295],
 ['Box office', 1548],
 ['Director', 0],
 ['Distributor', 357],
 ['Editor(s)', 548],
 ['Composer(s)', 518],
 ['Producer(s)', 202],
 ['Production company(s)', 1678],
 ['Writer(s)', 199],
 ['Genre', 6923],
 ['Original language(s)', 6875],
 ['Original network', 6908],
 ['Executive producer(s)', 6936],
 ['Production location(s)', 6986],
 ['Picture format', 6969],
 ['Audio format', 6972],
 ['Voices of', 7031],
 ['Followed by', 7024],
 ['Created by', 7023],
 ['Preceded by', 7023],
 ['Suggested by', 7032],
 ['alt_titles', 7012],
 ['Recorded', 7031],
 ['Venue', 7032],
 ['Label', 7031],
 ['Animation by', 7031],
 ['Color process', 7032],
 ['McCune–Reischauer', 7031],
 ['imdb_id', 0]]

In [41]:
# Using the map() method, we can see which values are not strings in the box_office data series
# 1 First, make a is_not_a_string() function:
def is_not_a_string(x):
    return type(x) != str
box_office[box_office.map(is_not_a_string)]


34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [42]:
# Alternative method is using a lambda function
# they use the following expression: lambda arguments: expression
# so he lambda function version of is_not_a_string() is:
# lambda x: type(x) != str
box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [43]:
# Next use a simple space as our joining character and apply the join() function only when our data points 
# are lists. The code looks like the following:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [44]:
# The box office data is written in two forms: "$123.4 million" (or billion), and "$123,456,789."
# Build a regular expression for each form, and then see what forms are left over.
# create the first form
# For the first form, our pattern match string will include six elements in the following order:
# 1 A dollar sign
# 2 An arbitrary (but non-zero) number of digits
# 3 An optional decimal point
# 4 An arbitrary (but possibly zero) number of more digits
# 5A space (maybe more than one)
# 6 The word "million" or "billion"
# Create a variable and  set it equal to the finished regular expression string. 

# 1 A dollar sign
# \$

# 2 An arbitrary (but non-zero) number of digits
# We'll add the \d character to specify digits only, and the + modifier to capture one or more digits. 
# Our regular expression string now appears as "\$\d+".

# 3 An optional decimal point
# Remember, the decimal point is a special character, so it needs to be escaped with a backslash. 
# Since the decimal point is optional, add a question mark modifier after it. Our regular expression 
# string now appears as "\$\d+\.?"

# 4 An arbitrary (but possibly zero) number of more digits
# we'll use the \d character to specify digits only, but now with the * modifier because there may be no 
# more digits after the decimal point. Our regular expression string now appears as "\$\d+\.?\d*"

# 5A space (maybe more than one)
# Now we're going to use the \s character to match whitespace characters. To be safe, we'll match any number 
# of whitespace characters with the * modifier. Our regular expression string now appears as "\$\d+\.?\d*\s*"

# 6 The word "million" or "billion"
# Since "million" and "billion" only differ by one letter, we can match it with a character set for the first 
# letter. We specify character sets with square brackets, so we'll add "[mb]illion" to the end of our string. 
# Our finished regular expression string now appears as "\$\d+\.?\d*\s*[mb]illion"

# Create a variable and  set it equal to the finished regular expression string. Because we need the 
# escape characters to remain, we need to preface the string with an r
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [45]:
# Now, to count up how many box office values match our first form. We'll use the str.contains() method on 
# box_office. To ignore whether letters are uppercase or lowercase, add an argument called flags, and set it 
# equal to re.IGNORECASE. In case the data is not a string, we'll add the na=False argument to parse the 
# non-string data to False. Finally, we can call the sum()method to count up the total number that return True. 
# Your code should look like the following:
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3896

In [46]:
# The box office data is written in two forms: "$123.4 million" (or billion), and "$123,456,789."
# Build a regular expression for each form, and then see what forms are left over.
# create the second form

# 1 Start with a dollar sign. we need to escape the dollar sign for it to match. Our regular expression string 
# starts like this: "\$".

# 2 Add a group of one to three digits.
# We'll use the \d character for digits, but this time, we'll modify it with curly brackets to only match one 
# through three repetitions. Our regular expression string now appears as "\$\d{1,3}".

# 3 Match at least one group starting with a comma and followed by exactly three digits
# To match a comma and exactly three digits, we'll use the string ",\d{3}". To match any repetition of that group, 
# we'll put it inside parentheses, and then put a plus sign after the parentheses: "(,\d{3})+". We'll add one more 
# modification to specify that this is a non-capturing group by inserting a question mark and colon after the opening 
# parenthesis: "(?:,\d{3})+". The use of a non-capturing group isn’t strictly necessary here, but it eliminates an 
# unwanted warning message in Jupyter Notebook. Our finished regular expression string now appears as "\$\d{1,3}(?:,\d{3})+"

# 4 Create a variable and  set it equal to the finished regular expression string.
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1544

In [47]:
# see which values aren't described by either. To be safe, we should see if any box office values are described by both.
# To make our code easier to understand, we'll create two Boolean Series called matches_form_one and matches_form_two, 
# and then select the box office values that don't match either. First, create the two Boolean series with the following 
# code:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [48]:
# Try the following code to see which values in box_office don't match either form
# Recall the Python logical keywords "not," "and," and "or." 
# this will throw an error!
# Comment out the code
# box_office[(not matches_form_one) and (not matches_form_two)]
# Instead, Pandas has element-wise logical operators:

#     The element-wise negation operator is the tilde: ~ (similar to "not")
#     The element-wise logical "and" is the ampersand: &
#     The element-wise logical "or" is the pipe: |
box_office[(~matches_form_one) & (~matches_form_two)]

34                         US$ 4,212,828
79                              $335.000
110                   $4.35-4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1070                          35,254,617
1147    $ 407,618 (U.S.) (sub-total) [1]
1446                        $ 11,829,959
1480                          £3 million
1611                            $520.000
1865                        ¥1.1 billion
2032                                 N/A
2091                                $309
2130               US$ 171.8 million [9]
2257                   US$ 3,395,581 [1]
2263            $ 1,223,034 ( domestic )
2347                            $282.175
2638            $ 104,883 (US sub-total)
2665         926,423 admissions (France)
2697      $ 1.7 million (US) (sub-total)
2823                            $414.000
2924                            $621.000
3088           $

In [49]:
# Fix Pattern Matches
# We can fix our pattern matches to capture more values by addressing these issues:

# 1 Some values have spaces in between the dollar sign and the number.
# 2 Some values use a period as a thousands separator, not a comma.
# 3 Some values are given as a range.
# 4 "Million" is sometimes misspelled as "millon."

# 1. Some values have spaces in between the dollar sign and the number.
# This is easy to fix. Just add \s* after the dollar signs. The new forms should look like the following:
# form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
# form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

In [50]:
# Fix Pattern Matches
# We can fix our pattern matches to capture more values by addressing these issues:
# 2 Some values use a period as a thousands separator, not a comma.
# This is slightly more complicated, but doable. Simply change form_two to allow for either a comma or period as a 
# thousands separator. We’d ordinarily do that by putting the comma and period inside straight brackets [,.], but 
# the period needs to be escaped with a slash [,\.]. The code should match the following:
# form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+'

# The results will also match values like 1.234 billion, but we're trying to change raw numbers like $123.456.789. 
# We don't want to capture any values like 1.234 billion, so we need to add a negative lookahead group that looks 
# ahead for "million" or "billion" after the number and rejects the match if it finds those strings. Don't forget 
# the space! The new form should look like this:
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [51]:
# Fix Pattern Matches
# We can fix our pattern matches to capture more values by addressing these issues:
# 3 Some values are given as a range.
# To solve this problem, we'll search for any string that starts with a dollar sign and ends with a hyphen, and then 
# replace it with just a dollar sign using the replace() method. The first argument in the replace() method is the 
# substring that will be replaced, and the second argument in the replace() method is the string to replace it with. 
# We can use regular expressions in the first argument by sending the parameter regex=True, as shown below.
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [52]:
# Fix Pattern Matches
# We can fix our pattern matches to capture more values by addressing these issues:
# 4 "Million" is sometimes misspelled as "millon."
# This is easy enough to fix; we can just make the second "i" optional in our match string with a question mark as follows:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

In [53]:
# Extract and Convert the Box Office Values
# Now that we've got expressions to match almost all the box office values, we'll use them to extract only the parts of 
# the strings that match. We do this with the str.extract() method. This method also takes in a regular expression string, 
# but it returns a DataFrame where every column is the data that matches a capture group. We need to make a regular 
# expression that captures data when it matches either form_one or form_two. We can do this easily with an f-string.

# The f-string f'{form_one}|{form_two}' will create a regular expression that matches either form_one or form_two, 
#so we just need to put the whole thing in parentheses to create a capture group. Our final string will be 
# f'({form_one}|{form_two})', and the full line of code to extract the data follows:
box_office.str.extract(f'({form_one}|{form_two})')

,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7070,$19.4 million
7071,$41.9 million
7072,$76.1 million
7073,$38.4 million


In [54]:
# Now we need a function to turn the extracted values into a numeric value. We'll call it parse_dollars, and parse_dollars 
# will take in a string and return a floating-point number. We'll start by making a skeleton function with comments 
# explaining each step, and then fill in the steps with actual code

# def parse_dollars(s):
    # if s is not a string, return NaN
    # if type(s) != str:
    #   return np.nan

    # if input is of the form $###.# million
   # if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"

        # convert to float and multiply by a million

        # return value

    # if input is of the form $###.# billion
 #   elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"

        # convert to float and multiply by a billion

        # return value

    # if input is of the form $###,###,###
  #  elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas

        # convert to float

        # return value

    # otherwise, return NaN
   # else:
    #    return np.nan

In [55]:
# Parse dollars continued: Now we need a function to turn the extracted values into a numeric value. 
# 1 Since we're working directly with strings, we'll use the re module to access the regular expression functions. We'll 
# use re.match(pattern, string) to see if our string matches a pattern. To start, we'll make some small alterations to 
# the forms we defined, splitting the million and billion matches from form one.

# 2 Next, we'll use re.sub(pattern, replacement_string, string) to remove dollar signs, spaces, commas, and letters, 
# if necessary.

# 3 Convert all the strings to floats, multiply by the right amount, and return the value.

def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [56]:
# Now we have everything we need to parse the box office values to numeric values.
# First, we need to extract the values from box_office using str.extract. Then we'll apply parse_dollars to the first 
# column in the DataFrame returned by str.extract, which in code looks like the following:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [57]:
# did not get same output as module. Goofed up a step. Ask at office hours tonight!
wiki_movies_df.sample(n=20)

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer,imdb_id,box_office
3178,https://en.wikipedia.org/wiki/Snow_Dogs_(film),2002,https://www.imdb.com/title/tt0281373/,Snow Dogs,"[Premise suggested by, Winterdance: The Fine M...","[Cuba Gooding Jr., James Coburn, Sisqó, Nichel...",NaN,Thomas E. Ackerman,"[January 18, 2002, (, 2002-01-18, )]",99 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0281373,115000000.0
5596,https://en.wikipedia.org/wiki/Good_Deeds,2012,https://www.imdb.com/title/tt1885265/,Good Deeds,NaN,"[Tyler Perry, Thandie Newton, Brian J. White, ...",NaN,Alexander Gruszynski,"[February 24, 2012, (, 2012-02-24, )]",111 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt1885265,35600000.0
3250,https://en.wikipedia.org/wiki/Big_Fish,2003,https://www.imdb.com/title/tt0319061/,Big Fish,"[Big Fish: A Novel of Mythic Proportions, by, ...","[Ewan McGregor, Albert Finney, Billy Crudup, J...",NaN,Philippe Rousselot,"[December 4, 2003, (, 2003-12-04, ), (New York...",125 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0319061,122900000.0
1253,https://en.wikipedia.org/wiki/The_Great_Mom_Swap,1995,https://www.imdb.com/title/tt0113215/,The Great Mom Swap,NaN,"[Shelley Fabares, Valerie Harper, Mary Kate Sc...",NaN,Sandi Sissel,"September 23, 1995",120 minutes (including commercials),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0113215,NaN
2730,https://en.wikipedia.org/wiki/Sexy_Beast,2000,https://www.imdb.com/title/tt0203119/,Sexy Beast,NaN,"[Ray Winstone, Ben Kingsley, Ian McShane, Aman...",NaN,Ivan Bird,"[13 September 2000, (, 2000-09-13, ), (, TIFF,...",88 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0203119,10200000.0
3011,https://en.wikipedia.org/wiki/Auto_Focus,2002,https://www.imdb.com/title/tt0298744/,Auto Focus,"[The Murder of Bob Crane, by, Robert Graysmith]","[Greg Kinnear, Willem Dafoe, Rita Wilson, Mari...",NaN,"[Jeffrey Greeley, Fred Murphy]","[October 18, 2002, (, 2002-10-18, )]",105 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0298744,2700000.0
2626,https://en.wikipedia.org/wiki/Dungeons_%26_Dra...,2000,https://www.imdb.com/title/tt0190374/,Dungeons & Dragons,NaN,"[Justin Whalin, Marlon Wayans, Thora Birch, Zo...",NaN,Douglas Milsome,"[December 8, 2000, (, 2000-12-08, )]",108 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0190374,33800000.0
4356,https://en.wikipedia.org/wiki/Captivity_(film),2007,https://www.imdb.com/title/tt0374563/,Captivity,NaN,"[Elisha Cuthbert, Daniel Gillies]",NaN,Daniel C. Pearl,"[13 July 2007, (, 2007-07-13, )]",85 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0374563,10900000.0
2001,https://en.wikipedia.org/wiki/The_Night_Flier_...,1997,https://www.imdb.com/title/tt0119784/,The Night Flier,"[The Night Flier, (short story) by, Stephen King]","[Miguel Ferrer, Julie Entwisle, Dan Monahan, M...",NaN,David Connell,"[May 2, 1997 (Italy), November 1997 (USA;, HBO...",93 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0119784,125397.0
546,https://en.wikipedia.org/wiki/Mac_(film),1992,https://www.imdb.com/title/tt0104783/,Mac,NaN,"[John Turturro, Michael Badalucco, Carl Capoto...",NaN,Ron Fortunato,"February 19, 1993 (USA)",117 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0104783,471120.0


In [58]:
# Run wiki_movies_df[‘box_office’]
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer,imdb_id,box_office
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987,21400000.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994,2700000.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005,57718089.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012,7331647.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018,6939946.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",NaN,Oliver Wood,"[December 25, 2018, (, 2018-12-25, ), (United ...",90 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt1255919,41900000.0
7072,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",NaN,Greig Fraser,"[December 11, 2018, (, 2018-12-11, ), (, Samue...",132 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6266538,76100000.0
7073,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",NaN,Michael Grady,"[November 8, 2018, (, 2018-11-08, ), (, AFI Fe...",120 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt4669788,38400000.0
7074,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",NaN,Julie Kirkwood,"[August 31, 2018, (, 2018-08-31, ), (, Telluri...",123 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt7137380,5500000.0
